In [1]:
import os
import math
import random
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchsummary

from torchvision import models
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from sklearn.utils import class_weight, shuffle
from sklearn.metrics import f1_score, fbeta_score, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
LR = 0.0001
SEED = 42
EPOCH = 15
BATCH_SIZE = 126
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train_dir = '/opt/ml/input/data/train'

os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
# torch.cuda.manual_seed_all(seed)  # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)

In [3]:
def preprocessedData():
    data_info = pd.read_csv(train_dir + '/train.csv', header=0)

    prepro_data_info = pd.DataFrame(columns={'id','img_path','race','mask','gender','age','label'})

    all_id, all_path, all_race, all_mask, all_age, all_gender, all_label = [],[],[],[],[],[],[]

    for absolute_path in glob(train_dir + "/images/*/*"):
        split_list = absolute_path.split("/")
        img_name = split_list[-1]
        img_path = split_list[-2]

        path_split = img_path.split("_")

        img_id = path_split[0]
        img_gender = 0 if path_split[1] == "male" else 1
        img_race = path_split[2]
        img_age = min(2, int(path_split[3]) // 30)

        img_mask = 0
        if 'incorrect' in img_name:
            img_mask = 1
        elif 'normal' in img_name:
            img_mask = 2

        all_id.append(img_id)
        all_path.append(absolute_path)
        all_race.append(img_race)
        all_mask.append(img_mask)
        all_gender.append(img_gender)
        all_age.append(img_age)
        all_label.append(img_mask*6 + img_gender*3 + img_age)

    prepro_data_info['id'] = all_id
    prepro_data_info['img_path'] = all_path
    prepro_data_info['race'] = all_race
    prepro_data_info['mask'] = all_mask
    prepro_data_info['gender'] = all_gender
    prepro_data_info['age'] = all_age
    prepro_data_info['label'] = all_label
    
    return prepro_data_info

In [4]:
class MyCustomDataset(Dataset):
    def __init__(self, prepro_data_info, transform, train=True):
        self.data_info = prepro_data_info # preprocessFunction()
        
        self.transform = transform
        self.image_path = self.data_info.img_path.tolist()
        self.label_arr = self.data_info.label.tolist()

        self.data_len = len(self.data_info.img_path)
        
        self.train = train

    def __getitem__(self, index):
        img_thing = Image.open(self.image_path[index])
        
        if self.transform:
            img_thing = self.transform(img_thing)
        
        if self.train:
            img_label = self.label_arr[index]
            return (img_thing, torch.tensor(img_label))
        else:
            return img_thing

    def __len__(self):
        return self.data_len

In [5]:
class MyCustomModel(nn.Module):
    
    def __init__(self):
        super(MyCustomModel, self).__init__()
        
        self.resnet18 = models.resnet18(pretrained=True)
        
        self.resnet18.fc = torch.nn.Linear(in_features=512, out_features=18, bias=True)
        
        # initialize
        nn.init.xavier_uniform_(self.resnet18.fc.weight)
        stdv = 1. / math.sqrt(self.resnet18.fc.weight.size(1))
        self.resnet18.fc.bias.data.uniform_(-stdv,stdv)
        
    def forward(self, x):
        x = self.resnet18(x)
        return x

In [6]:
transform = transforms.Compose([Resize((512,384),Image.BILINEAR), ToTensor(), Normalize(mean=(0.5,0.5,0.5), std=(0.2,0.2,0.2))])

train_data, valid_data = train_test_split(preprocessedData(), test_size=0.2)

my_custom_dataset = MyCustomDataset(train_data, transform)

In [7]:
my_model = MyCustomModel()
my_model.to(device)
print(my_model)

MyCustomModel(
  (resnet18): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

In [8]:
torchsummary.summary(my_model, (3, 512, 384), device="cuda")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 192]           9,408
       BatchNorm2d-2         [-1, 64, 256, 192]             128
              ReLU-3         [-1, 64, 256, 192]               0
         MaxPool2d-4          [-1, 64, 128, 96]               0
            Conv2d-5          [-1, 64, 128, 96]          36,864
       BatchNorm2d-6          [-1, 64, 128, 96]             128
              ReLU-7          [-1, 64, 128, 96]               0
            Conv2d-8          [-1, 64, 128, 96]          36,864
       BatchNorm2d-9          [-1, 64, 128, 96]             128
             ReLU-10          [-1, 64, 128, 96]               0
       BasicBlock-11          [-1, 64, 128, 96]               0
           Conv2d-12          [-1, 64, 128, 96]          36,864
      BatchNorm2d-13          [-1, 64, 128, 96]             128
             ReLU-14          [-1, 64, 

In [9]:
my_dataset_loader = torch.utils.data.DataLoader(dataset=my_custom_dataset, batch_size=BATCH_SIZE, shuffle=False)

valid_dataset = MyCustomDataset(valid_data, transform)

valid_dataloader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(my_model.parameters(), lr=LR)

# Start Training
for epoch in range(EPOCH):
    for i, (images, labels) in enumerate(my_dataset_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = my_model(images.float())
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        if i%10 == 0:
            print(f"Epoch: {epoch} | Batch ID: {i} | Loss: {loss.data}")
            
    # Validation Test
    targets = []
    all_predictions = []

    for images, labels in valid_dataloader:
        with torch.no_grad():
            images = images.to(device)
            labels = labels.to(device)

            pred = my_model(images.float())
            pred = pred.argmax(dim=-1)

            targets.extend(labels.cpu().numpy())

            all_predictions.extend(pred.cpu().numpy())

    print(f"Accuracy: {accuracy_score(targets, all_predictions)}")
    print(f"F1 Score: {np.mean(f1_score(targets, all_predictions, average=None))}")
            

print("done!")

Epoch: 0 | Batch ID: 0 | Loss: 3.3201913833618164
Epoch: 0 | Batch ID: 10 | Loss: 1.3203692436218262
Epoch: 0 | Batch ID: 20 | Loss: 0.8364559412002563
Epoch: 0 | Batch ID: 30 | Loss: 0.6236985921859741
Epoch: 0 | Batch ID: 40 | Loss: 0.32767900824546814
Epoch: 0 | Batch ID: 50 | Loss: 0.47867119312286377
Epoch: 0 | Batch ID: 60 | Loss: 0.24821555614471436
Epoch: 0 | Batch ID: 70 | Loss: 0.31142938137054443
Epoch: 0 | Batch ID: 80 | Loss: 0.2761459946632385
Epoch: 0 | Batch ID: 90 | Loss: 0.17847418785095215
Epoch: 0 | Batch ID: 100 | Loss: 0.2750527858734131
Epoch: 0 | Batch ID: 110 | Loss: 0.19567011296749115
Accuracy: 0.9354497354497354
F1 Score: 0.8331245014013057
Epoch: 1 | Batch ID: 0 | Loss: 0.1254342645406723
Epoch: 1 | Batch ID: 10 | Loss: 0.16760782897472382
Epoch: 1 | Batch ID: 20 | Loss: 0.16542555391788483
Epoch: 1 | Batch ID: 30 | Loss: 0.17309367656707764
Epoch: 1 | Batch ID: 40 | Loss: 0.08994191139936447
Epoch: 1 | Batch ID: 50 | Loss: 0.11845534294843674
Epoch: 1 | Ba

In [10]:
class MyTestDataset(Dataset):
    def __init__(self, data_info, transform, train=True):
        self.data_info = data_info
        
        self.transform = transform
        self.image_path = self.data_info.ImageID.tolist()

        self.data_len = len(self.image_path)
        
        self.train = train

    def __getitem__(self, index):
        img_thing = Image.open("/opt/ml/input/data/eval/images/" + self.image_path[index])
        
        if self.transform:
            img_thing = self.transform(img_thing)
        
        return img_thing

    def __len__(self):
        return self.data_len


test_df = pd.read_csv("/opt/ml/input/data/eval/info.csv")
    
    
test_dataset = MyTestDataset(test_df, transform = transform, train=False)

test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                batch_size=8,
                                                shuffle=False)

my_model.eval()

all_predictions = []
for images in test_dataloader:
    with torch.no_grad():
        images = images.to(device)
        pred = my_model(images.float())
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())


test_df['ans'] = all_predictions

test_df.to_csv("result4.csv", index=False)

print("done!")

done!
